# Multiple chains

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [2]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'Chicago, Illinois está en los Estados Unidos.'

In [3]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [4]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Pomegranate. and the flag of Nepal?')])

In [5]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of Dragonfruit is usually pink or red with green scales on the outside. The flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue with a green isosceles triangle on the hoist side.')

### Branching and Merging

In [6]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [7]:
chain.invoke({"input": "scrum"})

'In conclusion, while Scrum offers numerous benefits such as improved collaboration, transparency, adaptability, and productivity, it is important to acknowledge the potential drawbacks associated with this project management framework. The challenges of predictability, overemphasis on daily meetings, limited documentation, scope creep, dependency on team dynamics, and suitability for all projects should be carefully considered before implementing Scrum. It is essential to assess the specific needs and characteristics of a project to determine whether Scrum is the most appropriate approach for achieving successful outcomes. By addressing these potential cons proactively and tailoring the Scrum practices to fit the unique requirements of the project, teams can maximize the benefits of Scrum while mitigating its limitations.'